# Proeycto de IA - ChatBox

Departamento de Ciencias de la Computación, Universidad de Chile.

CC6409: Taller de Desarrollo de Poryectod e IA - Otoño 2023

**Integrantes:**
- Garrido Martín
- Gómez Nahuel
- Santelices Gustavo


## Import

In [35]:
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report

from sklearn.base import BaseEstimator, TransformerMixin
import re
import numpy as np
import nlpaug.augmenter.word as naw # To synonym, see  to more info https://github.com/makcedward/nlpaug
import nlpaug.augmenter.char as nac
from mtranslate import translate
import pickle
from gensim.models.keyedvectors import KeyedVectors
import os
from nltk.tokenize import TweetTokenizer

## data

In [2]:
# handmade
hand_nawel_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_nawel.csv", sep = ';')
hand_nawel_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_nawel.csv", sep = ';')
hand_nawel_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_nawel.csv", sep = ';')
hand_nawel_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_nawel.csv", sep = ';')

hand_tavo_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_tavo.csv", sep = ';')
hand_tavo_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_tavo.csv", sep = ';')
hand_tavo_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_tavo.csv", sep = ';')
hand_tavo_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_tavo.csv", sep = ';')

hand_martin_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_martin.csv", sep = ';', encoding = 'latin-1')
hand_martin_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_martin.csv", sep = ';')
hand_martin_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_martin.csv", sep = ';')
hand_martin_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_martin.csv", sep = ';')

In [3]:
df_ = pd.concat([hand_nawel_0, hand_nawel_1, hand_nawel_2, hand_nawel_3,
                 hand_tavo_0, hand_tavo_1, hand_tavo_2, hand_tavo_3,
                 hand_martin_0, hand_martin_1, hand_martin_2, hand_martin_3])

df_.count()

class    345
text     345
dtype: int64

In [4]:
df_['class'].value_counts()

class
0    101
1     89
2     87
3     68
Name: count, dtype: int64

## Data Augmentation (in a first step, to all parameters)

### ChatGPT sentences

In [5]:
# Nahuel
gpt_nawel_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/asistire_nawel.csv", sep = ';')
gpt_nawel_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/no_asistire_nawel.csv", sep = ';')
gpt_nawel_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/reagendar_nawel.csv", sep = ';')
gpt_nawel_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/pedir_nawel.csv", sep = ';')

# Martin
gpt_martin_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/asistire_martin.csv", sep = ';')
gpt_martin_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/no_asistire_martin.csv", sep = ';')
gpt_martin_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/reagendar_martin.csv", sep = ';')
gpt_martin_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/pedir_martin.csv", sep = ';')

# Gustavo
#gpt_tavo_0 = pd.read_csv("", sep = ';')
#gpt_tavo_1 = pd.read_csv("", sep = ';')
#gpt_tavo_2 = pd.read_csv("", sep = ';')
#gpt_tavo_3 = pd.read_csv("", sep = ';')

In [6]:
# Concat the chatgpt sentences
df_ = pd.concat([df_, 
                 gpt_nawel_0, gpt_nawel_1, gpt_nawel_2, gpt_nawel_3,
                 gpt_martin_0, gpt_martin_1, gpt_martin_2, gpt_martin_3])

In [7]:
df_['class'].value_counts()

class
0    185
1    178
2    162
3    141
Name: count, dtype: int64

### Translate augmentation (functions)

In [8]:
# Extra : Translate Ideas
def sp_to_en(data):
    augmented_data = []
    for sentence in data:
        # Translate Spanish sentence to English
        translated_sentence = translate(sentence, "en")
        augmented_data.append(translated_sentence)

    return augmented_data

def en_to_sp(data):
    augmented_data = []
    for sentence in data:
        # Translate English sentence to Spanish
        translated_sentence = translate(sentence, "es")

        augmented_data.append(translated_sentence)

    return augmented_data

In [9]:
# Import model
aug_fb = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en'
)

In [10]:
def Translate_aug(df_):
    # Getting the values with two or more words.
    df_english_aug = df_[df_['text'].apply(lambda x: len(x.split()) > 2)]

    # Traslate to English
    l_english_aug = sp_to_en(df_english_aug.text.tolist())

    # Find Synonyms
    english_syn_aug = aug_fb.augment(l_english_aug)

    # Translate to spanish
    l_english_aug = en_to_sp(df_english_aug.text.tolist())

    # Append text with label
    df_english = pd.DataFrame()
    df_english['class'] = df_english_aug['class']
    df_english['text'] = l_english_aug

    return df_english

### Data aug in spanish (functions)

In [11]:
# Wordnet - spa
aug_wn = naw.SynonymAug(aug_src='wordnet', lang='spa')

In [12]:
def spanish_aug(df_):
    # Getting the values with ten or more words.
    df_spanish_aug = df_[df_['text'].apply(lambda x: len(x.split()) > 10)]
    l_augmented = aug_wn.augment(df_spanish_aug.text.tolist())

    # Append text with label
    df_spanish = pd.DataFrame()
    df_spanish['class'] = df_spanish_aug['class']
    df_spanish['text'] = l_augmented

    return df_spanish

## Custom Features

In [13]:
class PreProccesingTransformer(BaseEstimator, TransformerMixin):
    def preprocess(self,sentence):
      # Deleting all except: exclamation/question signs and accents
      new_word = re.sub(r"[^a-zA-ZáéíóúÁÉÍÓÚñÑ¡!¿?\s]", '', sentence)
      # Deleting double blank spaces
      new_sentence = new_word.replace('  ',' ').replace('\n','').strip()
      return new_sentence

    def transform(self, X, y=None):
        values = []
        for tweet in X:
            values.append(self.preprocess(tweet))

        return(np.array(values))

    def fit(self, X, y=None):
        return self

In [27]:
class Doc2VecTransformer_auxiliar(BaseEstimator, TransformerMixin):    
    def __init__(self, model, tokenizer,aggregation_func):
        self.model = model
        self.tokenizer=tokenizer
        # agregation func (np.min, np.max, np.mean, np.sum, ...)
        self.aggregation_func = aggregation_func
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        doc_embeddings = []
        for doc in X:
            # tokenization the doc
            tokens = self.tokenizer.tokenize(doc)
            selected_wv = []
            for token in tokens:
                if token in self.model.index_to_key:
                    selected_wv.append(self.model[token])
                
            if len(selected_wv) > 0:
                doc_embedding = self.aggregation_func(np.array(selected_wv), axis=0)
                doc_embeddings.append(doc_embedding)
            else: 
                doc_embeddings.append(np.zeros(self.model.vector_size))

        return np.array(doc_embeddings)

In [32]:
current_dir = os.getcwd()
wordvectors_file_vec = os.path.join(current_dir, "WordEmbeddings", "Spanis_Unannotated_Corpora.vec")

cantidad = 600000
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

## Pipeline

### Run Pipeline

In [14]:
# Defining threshold
thresholds = {
    0: 0.90,  # class 0
    1: 0.80,  # class 1
    2: 0.80,   # class 2
    3: 0.80   #class 3
}


def custom_predict(model, X, class_thresholds):
    # Get probabilities
    probabilities = model.predict_proba(X)

    # Aplied threshold
    modified_predictions = np.argmax(probabilities, axis=1)
    for class_label, threshold in class_thresholds.items():
        modified_predictions[probabilities[:, class_label] > threshold] = class_label

    return modified_predictions

In [15]:
def run(dataset, pipeline):

    X_train, X_test, y_train, y_test = train_test_split(
        dataset['text'],
        dataset['class'],
        shuffle=True,
        test_size=0.2,
        random_state=42,
        stratify=dataset['class']
    )

    print(f"# Len Training Data: {len(X_train)}")
    print(f"# Len Testing Data: {len(X_test)}")

    print('Aplying data aug')

    df_to_aug = pd.DataFrame({'text': X_train, 'class': y_train})

    english_df = Translate_aug(df_to_aug)
    spanish_df = spanish_aug(df_to_aug)

    X_train = pd.concat([X_train, english_df['text'], spanish_df['text']])
    y_train = pd.concat([y_train, english_df['class'], spanish_df['class']])

    print(f"# NEW Len Training Data: {len(X_train)}")
    print(f"# NEW Len Testing Data: {len(X_test)}")

    pipeline.fit(X_train, y_train)


    # Classes propbailities
    probabilities = pipeline.predict_proba(X_test)


    # Predicted labels
    predicted_labels = pipeline.predict(X_test)
    #predicted_labels = custom_predict(pipeline, X_test, thresholds)


    print(classification_report(y_test, predicted_labels))


    return pipeline, X_train, X_test, y_train, y_test, predicted_labels, probabilities


## Defining Pipelines

In [37]:
# simple Pipeline only BOW

def get_experiment_0_pipeline():

    return Pipeline(
        [
            (
                "features",
                FeatureUnion(
                    [
                        ("bow", CountVectorizer()),
                    ]
                ),
            ),
            ("clf", MultinomialNB()),
        ]
    )

In [17]:
# simple Pipeline only Preprocessing + BOW

def get_experiment_1_pipeline():

    return Pipeline(
        [
            ("preprocessing", PreProccesingTransformer()),
            (
                "features",
                FeatureUnion(
                    [
                        ("bow", CountVectorizer())
                    ]
                ),
            ),
            ("clf", MultinomialNB()),
        ]
    )

In [45]:
# simple Pipeline only BOW

def get_experiment_2_pipeline():

    return Pipeline(
        [
            (
                "features",
                FeatureUnion(
                    [
                        ("bow", CountVectorizer()),
                        ('d2v', Doc2VecTransformer_auxiliar(wordvectors,TweetTokenizer(),np.sum)),
                    ]
                ),
            ),
           ("clf", GradientBoostingClassifier(n_estimators=200, learning_rate=0.7, max_depth=1, random_state=0)),
        ]
    )

In [46]:
# simple Pipeline only BOW

def get_experiment_3_pipeline():

    return Pipeline(
        [
            (
                "features",
                FeatureUnion(
                    [
                        ("bow", CountVectorizer()),
                        ('d2v', Doc2VecTransformer_auxiliar(wordvectors,TweetTokenizer(),np.sum)),
                    ]
                ),
            ),
           ('clf', SVC(probability=True)),
        ]
    )

## Implementación del modelo

In [38]:
# Simple Pipeline Test
pipeline0 = get_experiment_0_pipeline()

prob = run(df_, pipeline0)

# Len Training Data: 532
# Len Testing Data: 134
Aplying data aug
# NEW Len Training Data: 1181
# NEW Len Testing Data: 134
              precision    recall  f1-score   support

           0       0.94      0.78      0.85        37
           1       0.84      0.86      0.85        36
           2       0.82      0.94      0.87        33
           3       0.93      0.93      0.93        28

    accuracy                           0.87       134
   macro avg       0.88      0.88      0.88       134
weighted avg       0.88      0.87      0.87       134



In [20]:
# Save the model
with open('model0.bin', 'wb') as fil:
    pickle.dump(pipeline0, fil)

In [22]:
# Simple Pipeline Test (+ preprocessing)
pipeline1 = get_experiment_1_pipeline()

_ = run(df_, pipeline1)

# Len Training Data: 532
# Len Testing Data: 134
Aplying data aug
# NEW Len Training Data: 1181
# NEW Len Testing Data: 134
              precision    recall  f1-score   support

           0       0.94      0.78      0.85        37
           1       0.83      0.83      0.83        36
           2       0.79      0.94      0.86        33
           3       0.93      0.93      0.93        28

    accuracy                           0.87       134
   macro avg       0.87      0.87      0.87       134
weighted avg       0.87      0.87      0.87       134



In [23]:
# Save the model
with open('model1.bin', 'wb') as fil:
    pickle.dump(pipeline1, fil)

In [48]:
# Word 2 vec text
from sklearn.svm import SVC
pipeline2 = get_experiment_2_pipeline()

prob = run(df_, pipeline2)

# Len Training Data: 532
# Len Testing Data: 134
Aplying data aug
# NEW Len Training Data: 1181
# NEW Len Testing Data: 134
              precision    recall  f1-score   support

           0       0.87      0.92      0.89        37
           1       0.91      0.89      0.90        36
           2       0.87      0.79      0.83        33
           3       0.90      0.96      0.93        28

    accuracy                           0.89       134
   macro avg       0.89      0.89      0.89       134
weighted avg       0.89      0.89      0.89       134



In [49]:
from sklearn.ensemble import GradientBoostingClassifier
pipeline3 = get_experiment_3_pipeline()

prob = run(df_, pipeline3)

# Len Training Data: 532
# Len Testing Data: 134
Aplying data aug
# NEW Len Training Data: 1181
# NEW Len Testing Data: 134
              precision    recall  f1-score   support

           0       0.76      0.86      0.81        37
           1       0.89      0.69      0.78        36
           2       0.83      0.91      0.87        33
           3       0.93      0.93      0.93        28

    accuracy                           0.84       134
   macro avg       0.85      0.85      0.85       134
weighted avg       0.85      0.84      0.84       134



In [24]:
## User response

In [25]:
map_label= {
    0: 'Asistir a la',
    1: 'Cancelar la',
    2: 'Reagendar la',
    3: 'Pedir una'
}

options = {
    0: 'Su reserva a quedado confirmada para el DD de MM',
    1: 'Su reserva con fecha DD de MM a quedado cancelada',
    2: 'Las fechas para reagendar son las siguientes',
    3: 'Las fechas próximas son las siguientes'
}

def manage_mesages(target):
    # Make a prediction
    label = pipeline1.predict([target])[0]
    print(f"usted quiere {map_label[label]} hora")
    print(options[label])

In [26]:
manage_mesages('Puedo cancelar la hora??')

usted quiere Cancelar la hora
Su reserva con fecha DD de MM a quedado cancelada
